In [1]:
import pandas as pd

pd.options.display.max_colwidth = 300

In [2]:
# Excel file path of the matrix I
file_path = 'Close Neighbour Algorithm.xlsx'

# Read the Excel file
matrix_I = pd.read_excel(file_path, sheet_name="Example1", index_col=0)
# matrix_I = pd.read_excel(file_path, sheet_name="Exercise2", index_col=0)
# matrix_I = pd.read_excel(file_path, sheet_name="Example4", index_col=0)

matrix_I.fillna(0, inplace=True)

# Display matrix I
matrix_I.astype(int)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
1,0,1,1,0,0,0,0,1,1,0,1,0,1,1,0,1,1,0,1,0
2,0,0,1,1,0,1,1,0,0,0,0,0,0,1,0,0,0,1,0,1
3,0,1,0,0,0,0,0,1,1,0,1,0,1,1,0,1,1,0,1,0
4,0,0,1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,1
5,1,0,0,0,1,1,0,0,0,1,0,1,0,0,1,0,1,0,0,0
6,1,0,0,0,1,0,0,0,1,1,0,1,0,0,1,0,0,0,0,1
7,0,0,1,1,0,1,1,0,0,0,1,1,0,0,0,0,0,1,0,1
8,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1


# Stage 1

In [4]:
# Compose matrix B: number of components common to each pair of machines
matrix_B = pd.DataFrame()
for index, row in matrix_I.iterrows():
    # for matrix of 0s and 1s, common means the sum of the multiplication of both rows = 1
    matrix_B[index] = matrix_I.apply(lambda row: (matrix_I.iloc[index-1] * row).sum(), axis=1)
    # for Bii, remove values
    matrix_B.loc[index, index] = 0

# Compose Si column by summing the values of each row
matrix_B["Si"] = matrix_B.sum()

# Display matrix B
matrix_B.astype(int)

,1,2,3,4,5,6,7,8,Si
1,0,2,9,1,1,1,2,1,17
2,2,0,1,6,1,1,6,6,23
3,9,1,0,0,1,1,1,0,13
4,1,6,0,0,2,2,6,6,23
5,1,1,1,2,0,5,2,1,13
6,1,1,1,2,5,0,2,1,13
7,2,6,1,6,2,2,0,6,25
8,1,6,0,6,1,1,6,0,21


In [5]:
# use the machine with Max Si value; lower index first if tie in Si value
current_machine = matrix_B["Si"].idxmax()
# selected machines
new_machine_order = [current_machine]
# A mirror of matrix B, which reduced columns as the selection progress
matrix_B_reduced = matrix_B.copy()
#    Remove column of the first selected machine
matrix_B_reduced.drop(current_machine, axis=1, inplace=True)

# run per machine
while len(new_machine_order) < matrix_B.shape[0]:
    # machine of selected machine in matrix B
    current_machine_B = matrix_B_reduced.loc[current_machine]
    print(f"current machine {current_machine}:\n{current_machine_B.to_frame().T}")
    machine = current_machine_B.drop("Si")
    machines_bij = machine.loc[machine==machine.max()].index.tolist()
    print(f"potential machines: {machines_bij}")

    if len(machines_bij) == 1:
        current_machine = machines_bij[0]
        print(f"selected machine: {current_machine}")
        print(f"selected reason: biggest Bij")
    else:
        si_values = matrix_B_reduced["Si"][machines_bij]
        # print(si_values)
        print(f"Si values:\n{si_values.to_frame().T}")
        machines_si = si_values.loc[si_values==si_values.max()].index.tolist()
        if len(machines_si) == 1:
            current_machine = machines_si[0]
            print(f"selected machine: {current_machine}")
            print(f"selected reason: Bij tie; greater Si value")
        else:
            current_machine = machines_si[0]
            print(f"selected reason: Bij tie; Si tie; lower i")
    new_machine_order.append(current_machine)
    matrix_B_reduced.drop(current_machine, axis=1, inplace=True)
    print(f"selected machines: {new_machine_order} \n\n")

current machine 7:
     1    2    3    4    5    6    8    Si
7  2.0  6.0  1.0  6.0  2.0  2.0  6.0  25.0
potential machines: [2, 4, 8]
Si values:
       2     4     8
Si  23.0  23.0  21.0
selected reason: Bij tie; Si tie; lower i
selected machines: [7, 2] 


current machine 2:
     1    3    4    5    6    8    Si
2  2.0  1.0  6.0  1.0  1.0  6.0  23.0
potential machines: [4, 8]
Si values:
       4     8
Si  23.0  21.0
selected machine: 4
selected reason: Bij tie; greater Si value
selected machines: [7, 2, 4] 


current machine 4:
     1    3    5    6    8    Si
4  1.0  0.0  2.0  2.0  6.0  23.0
potential machines: [8]
selected machine: 8
selected reason: biggest Bij
selected machines: [7, 2, 4, 8] 


current machine 8:
     1    3    5    6    Si
8  1.0  0.0  1.0  1.0  21.0
potential machines: [1, 5, 6]
Si values:
       1     5     6
Si  17.0  13.0  13.0
selected machine: 1
selected reason: Bij tie; greater Si value
selected machines: [7, 2, 4, 8, 1] 


current machine 1:
     3    5 

In [6]:
matrix_intermediate = matrix_I.reindex(index=new_machine_order)
matrix_intermediate.astype(int).astype(str).replace("0", "")

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
7,,,1,1,,1,1,,,,1,1,,,,,,1,,1
2,,,1,1,,1,1,,,,,,,1,,,,1,,1
4,,,1,1,,1,1,,,1,,,,,,,,1,,1
8,,,1,1,,1,1,,,,,,,,,,,1,,1
1,,1,1,,,,,1,1,,1,,1,1,,1,1,,1,
3,,1,,,,,,1,1,,1,,1,1,,1,1,,1,
5,1,,,,1,1,,,,1,,1,,,1,,1,,,
6,1,,,,1,,,,1,1,,1,,,1,,,,,1


# Stage 2

## SEQ1 and SEQ2 method

In [9]:
def stage_2_seq1_seq2(matrix_intermediate):
    matrix_intermediate_reduced = matrix_intermediate.copy()
    new_part_order = []
    counter = 1
    while matrix_intermediate_reduced.shape[0] > 1 and len(new_part_order)<matrix_intermediate.shape[1]:
        if matrix_intermediate_reduced.shape[0] % 2 == 1:
            # odd
            split = matrix_intermediate_reduced.shape[0] // 2
            if split % 2 == 0:
                # make SEQ1 to have odd rows
                split = split + 1
        else:
            # even
            split = matrix_intermediate_reduced.shape[0] // 2
        
        seq1 = matrix_intermediate_reduced.iloc[0:split]
        seq2 = matrix_intermediate_reduced.iloc[split:matrix_intermediate_reduced.shape[0]]
        print(f"Phase {counter}")
        print(f"SEQ1")
        display(seq1.astype(int))
        print("-----")
        display(seq2.astype(int))
        print(f"SEQ2\n")
        
        seq1_on_seq2 = seq1.sum() > seq2.sum()
        display(seq1.sum().astype(int).to_frame().T)
        display(seq2.sum().astype(int).to_frame().T)
        display(seq1_on_seq2.to_frame().T)
        products_greater = seq1_on_seq2[seq1_on_seq2==True].index.tolist()
        print(f"greater products: {products_greater}")
        if len(products_greater) > 0:
            new_part_order = new_part_order + products_greater
            print(f"selected products {new_part_order}")
        else:
            products_remains = seq1_on_seq2.index.tolist()
            print(f"No products are greater: {products_remains}")
            new_part_order = new_part_order + products_remains
            print(f"selected products {new_part_order}")
        matrix_intermediate_reduced = seq2.copy()
        matrix_intermediate_reduced.drop(products_greater, axis=1, inplace=True)
        counter += 1
        print("\n")
        # print(f"debug: product remains: {matrix_intermediate_reduced.columns}")
        # print(f"debug: product remains index: {matrix_intermediate_reduced.index}")
        # break

    products_remains = matrix_intermediate_reduced.columns.tolist()
    print(f"No products are greater: {products_remains}")
    new_part_order = new_part_order + products_remains
    print(f"selected products {new_part_order}")
    
    # Sanity check
    if matrix_intermediate.shape[1] != len(new_part_order):
        print(f"Error: matrix with {matrix_intermediate.shape[1]} columns but new part order only got {new_part_order} entries")

    return new_part_order

In [10]:
new_part_order = stage_2_seq1_seq2(matrix_intermediate)
matrix_solution = matrix_intermediate.reindex(columns=new_part_order)
matrix_solution.astype(int).astype(str).replace("0", "")

Phase 1
SEQ1


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
7,0,0,1,1,0,1,1,0,0,0,1,1,0,0,0,0,0,1,0,1
2,0,0,1,1,0,1,1,0,0,0,0,0,0,1,0,0,0,1,0,1
4,0,0,1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,1
8,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1


-----


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
1,0,1,1,0,0,0,0,1,1,0,1,0,1,1,0,1,1,0,1,0
3,0,1,0,0,0,0,0,1,1,0,1,0,1,1,0,1,1,0,1,0
5,1,0,0,0,1,1,0,0,0,1,0,1,0,0,1,0,1,0,0,0
6,1,0,0,0,1,0,0,0,1,1,0,1,0,0,1,0,0,0,0,1


SEQ2



...

,1,5,10,12,15
0,1,1,1,1,1


,1,5,10,12,15
0,1,1,1,1,1


,1,5,10,12,15
0,False,False,False,False,False


greater products: []
No products are greater: [1, 5, 10, 12, 15]
selected products [3, 4, 6, 7, 18, 20, 2, 8, 9, 11, 13, 14, 16, 17, 19, 1, 5, 10, 12, 15]


No products are greater: [1, 5, 10, 12, 15]
selected products [3, 4, 6, 7, 18, 20, 2, 8, 9, 11, 13, 14, 16, 17, 19, 1, 5, 10, 12, 15, 1, 5, 10, 12, 15]
Error: matrix with 20 columns but new part order only got [3, 4, 6, 7, 18, 20, 2, 8, 9, 11, 13, 14, 16, 17, 19, 1, 5, 10, 12, 15, 1, 5, 10, 12, 15] entries


,3,4,6,7,18,20,2,8,9,11,...,1,5,10,12,15,1,5,10,12,15
7,1,1,1,1,1,1,,,,1,...,,,,1,,,,,1,
2,1,1,1,1,1,1,,,,,...,,,,,,,,,,
4,1,1,1,1,1,1,,,,,...,,,1,,,,,1,,
8,1,1,1,1,1,1,,,,,...,,,,,,,,,,
1,1,,,,,,1,1,1,1,...,,,,,,,,,,
3,,,,,,,1,1,1,1,...,,,,,,,,,,
5,,,1,,,,,,,,...,1,1,1,1,1,1,1,1,1,1
6,,,,,,1,,,1,,...,1,1,1,1,1,1,1,1,1,1


## NNOV and NNOW Method

In [12]:
def max_consecutive_ones_from_0(v_seq):
    max_count = 0
    count = 0
    for i, value in enumerate(v_seq):
        if (v_seq[i] == 1):
            count += 1
        else:
            break
    return count

def max_consecutive_ones_subset(w_seq):
    max_count = 0
    count = 0
    for i, value in enumerate(w_seq):
        if (w_seq[i] == 1):
            count += 1
            max_count = max(max_count, count)
        else:
            count = 0
    return max_count

def stage_2_nnov_nnow(matrix_intermediate):
    matrix_intermediate_reduced = matrix_intermediate.copy()
    new_part_order = []
    counter = 1
    v = 0
    is_printing = False
    while matrix_intermediate_reduced.shape[0] > 1 and len(new_part_order)<matrix_intermediate.shape[1]:
        # print(f"Phase {counter}")
        # display(matrix_intermediate_reduced)
        # NNOV = matrix_intermediate_reduced.reset_index(drop=True).iloc[v:matrix_intermediate_reduced.shape[0]].apply(lambda col: col[col==0].index[0], axis=0)
        NNOV = matrix_intermediate_reduced.reset_index(drop=True).iloc[v:matrix_intermediate_reduced.shape[0]].apply(
            lambda col: max_consecutive_ones_from_0(col), axis=0
        )
        # display(NNOV.to_frame().T)
        NNOW = pd.Series()
        for part in matrix_intermediate_reduced.columns:
            w = NNOV[part] + v 
            w_seq = matrix_intermediate_reduced[part].reset_index(drop=True).iloc[w:matrix_intermediate_reduced.shape[0]].reset_index(drop=True)
            # display(w_seq.to_frame().T)
            NNOW.loc[part] = max_consecutive_ones_subset(w_seq)
        # display(NNOW.astype(int).to_frame().T)
    
        NNOV_on_NNOW = NNOV > NNOW
        # display(NNOV_on_NNOW.to_frame().T)
        parts_greater = NNOV_on_NNOW[NNOV_on_NNOW==True].index.tolist()
        # print(f"greater parts: {parts_greater}")
        if len(parts_greater) > 0:
            new_part_order = new_part_order + parts_greater
            # print(f"selected parts {new_part_order}")
            is_printing = True
        else:
            is_printing = False

        if is_printing:
            print(f"Phase {counter}")
            display(matrix_intermediate_reduced)
            display(NNOV.to_frame().T)
            display(NNOW.astype(int).to_frame().T)
            display(NNOV_on_NNOW.to_frame().T)
            print(f"greater parts: {parts_greater}")
            print(f"selected parts {new_part_order}")
            print("\n")
        matrix_intermediate_reduced = matrix_intermediate_reduced.iloc[v+1:matrix_intermediate_reduced.shape[0]]
        matrix_intermediate_reduced.drop(parts_greater, axis=1, inplace=True)
        counter += 1
        # print("\n")
        # break

    # add remaining parts to the list
    # parts_remains = NNOV_on_NNOW.index.tolist()
    parts_remains = matrix_intermediate_reduced.columns.tolist()
    print(f"No parts are greater: {parts_remains}")
    new_part_order = new_part_order + parts_remains
    print(f"selected parts {new_part_order}")
    
    # Sanity check
    if matrix_intermediate.shape[1] != len(new_part_order):
        print(f"Error: matrix with {matrix_intermediate.shape[1]} columns but new part order only got {new_part_order} entries")

    return new_part_order

In [13]:
# new_part_order = stage_2_nnov_nnow(matrix_intermediate)
# matrix_solution = matrix_intermediate.reindex(columns=new_part_order)
# matrix_solution.astype(int).astype(str).replace("0", "")

# Grouping

In [15]:
def calc_group_efficiency(df, groups):
    # count in-cellular elements
    spaces_occupied = 0
    spaces_within_cell = 0
    group_prev = (0, 0)
    for group in groups:
        spaces_occupied += df.iloc[group_prev[0]:group[0], group_prev[1]:group[1]].sum().sum()
        spaces_within_cell += (group[0]-group_prev[0])*(group[1]-group_prev[1])
        group_prev = group
    incidences_outside_cells = df.sum().sum() - spaces_occupied
    
    
    Ex = incidences_outside_cells / matrix_solution_grouping.sum().sum()
    In = spaces_occupied / spaces_within_cell
    print(f"incidences_outside_cells: {incidences_outside_cells}")
    print(f"overall_cells: {matrix_solution_grouping.sum().sum()}")
    print(f"incidences_outside_cells: {spaces_occupied}")
    print(f"incidences_outside_cells: {spaces_within_cell}")
    
    GE = ((1-Ex) + In)/2.0
    return GE

In [16]:
matrix_solution_grouping = matrix_solution.copy().astype(int)
# highlight spaces as True
matrix_solution_spaces = matrix_solution_grouping==0

# Define your Groups here
# Example 1
groups = [(4, 6), (6, 15)]
# Example 3
# groups = [(3,4)]
# Exercise 1
# groups = [(5,5)]
# Exercise 2
# groups = [(2,2), (5,5)]
groups.append((matrix_solution_grouping.shape[0], matrix_solution_grouping.shape[1]))
GE = calc_group_efficiency(matrix_solution_grouping, groups)
GE

incidences_outside_cells: 11
overall_cells: 73
incidences_outside_cells: 62
incidences_outside_cells: 62


0.9246575342465754